In [4]:
import pandas as pd
import numpy as np
import os
import csv
import ast
from collections import Counter
import geopandas as gpd
import h5py
from datetime import datetime, timedelta
#### personal packages####
import sys

In [5]:
cluster_name = {0: 'Glocery&Pharmacies', 1: 'Retails', 2: 'Arts&Entertainment', 3: 'Restaurants&Bars',
                4: 'Educations', 5: 'Healthcares', 6: 'others'}

In [6]:
df_flow = pd.read_csv('../results/outflow_7c_wk_raw_loss_visitors.csv')
df_flow['MODZCTA'] = df_flow['MODZCTA'].astype(str)

In [7]:
cluster_cols = [cluster_name[0]+'_{}'.format(i) for i in range(56)] 
# 56 weeks from 2020-02-04 to 2021-03-02
# only need 53 weeks from 2020-02-24 to 2021-02-02
df_vist_cluster_i = df_flow[['MODZCTA','mod_idx']+cluster_cols].copy()

final_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA','mod_idx'], value_vars = cluster_cols[3:3+51], var_name='week', value_name=cluster_name[0]+'_visits_weekly')
#### 
# Correctly adjust the code to perform the intended replacement
final_df['week'] = final_df['week'].apply(lambda x: (int(x.split('_')[1]) - 3))

for c_idx in range(1,7):
    print(cluster_name[c_idx])
    cluster_cols = [cluster_name[c_idx]+'_{}'.format(i) for i in range(56)]
    df_vist_cluster_i = df_flow[['MODZCTA', 'mod_idx']+cluster_cols].copy()
    temp_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA', 'mod_idx'], value_vars = cluster_cols[3:3+51], 
                    var_name='week', value_name=cluster_name[c_idx]+'_visits_weekly')
    temp_df['week'] = temp_df['week'].apply(lambda x: (int(x.split('_')[1]) - 3))
    final_df = pd.merge(final_df, temp_df, on=['MODZCTA','mod_idx','week'])

Retails
Arts&Entertainment
Restaurants&Bars
Educations
Healthcares
others


In [8]:
### 1 covid_data_population file 
df_mod_borough = pd.read_csv('../Data/data-by-modzcta_11102020.csv')
# the len of the df_mod_borough is 177, the number of MODZCTA is 177
df_mod_borough['MODIFIED_ZCTA'] = df_mod_borough['MODIFIED_ZCTA'].astype(str)
replace_dic = dict(zip(['Manhattan', 'Staten Island', 'Bronx', 'Queens', 'Brooklyn'],['MN','SI','BX','QN','BK']))
df_mod_borough['borough'] = df_mod_borough['BOROUGH_GROUP'].replace(replace_dic)

for col in ['COVID_CASE_COUNT','COVID_DEATH_COUNT']:
    df_mod_borough[col] = df_mod_borough[col].astype(float)
    df_mod_borough.loc[df_mod_borough[col]==0,col] = 0.1
    df_mod_borough[col+'_log'] = np.log(df_mod_borough[col])

df_merged = final_df.merge(df_mod_borough, right_on = 'MODIFIED_ZCTA', left_on='MODZCTA')
columns_to_delete = ['MODIFIED_ZCTA','BOROUGH_GROUP']
df_merged = df_merged.drop(columns=columns_to_delete)
df_merged['MODZCTA'] = df_merged['MODZCTA'].astype(int)

In [18]:
df_mod_borough

,MODIFIED_ZCTA,NEIGHBORHOOD_NAME,BOROUGH_GROUP,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,TOTAL_COVID_TESTS,borough,COVID_CASE_COUNT_log,COVID_DEATH_COUNT_log
0,10001,Chelsea/NoMad/West Chelsea,Manhattan,500.0,1810.74,27613.09,27.0,97.78,4.02,11020,MN,6.214608,3.295837
1,10002,Chinatown/Lower East Side,Manhattan,1525.0,2024.62,75322.71,160.0,212.42,5.66,26793,MN,7.329750,5.075174
2,10003,East Village/Gramercy/Greenwich Village,Manhattan,763.0,1413.54,53977.81,35.0,64.84,2.68,27764,MN,6.637258,3.555348
3,10004,Financial District,Manhattan,59.0,1985.12,2972.12,1.0,33.65,3.44,1598,MN,4.077537,0.000000
4,10005,Financial District,Manhattan,125.0,1427.39,8757.23,2.0,22.84,3.30,3455,MN,4.828314,0.693147
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,11691,Edgemere/Far Rockaway,Queens,3209.0,4799.85,66856.31,379.0,566.89,12.79,24589,QN,8.073715,5.937536
173,11692,Arverne/Edgemere,Queens,760.0,3614.03,21029.13,95.0,451.75,10.56,7118,QN,6.633318,4.553877
174,11693,Arverne/Broad Channel,Queens,397.0,3132.86,12672.14,29.0,228.85,10.20,3744,QN,5.983936,3.367296
175,11694,Belle Harbor-Neponsit/Rockaway Park,Queens,864.0,4157.96,20779.40,90.0,433.12,11.19,7593,QN,6.761573,4.499810


In [9]:
len(df_merged)

9027

Merge all the other data at the zipcode level

In [9]:
### 2 read scores data at zipcode level#########
df_mod_score = pd.read_csv('../results/modzcta_scores.csv')
df_2 = df_merged.merge(df_mod_score, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
df_2.dropna(subset=['score_median'],inplace=True)
print(len(df_2))

6477


In [10]:
##### 3 social economic data ######
# df_sed_nyc = pd.read_csv('../Data/sed_nyc_modzcta.csv')
df_sed_nyc = pd.read_csv('../Data/sed_old.csv')
df_3 = df_2.merge(df_sed_nyc, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
print(len(df_3))

6477


In [11]:
### 4 merge with borough cases ###
covid_cases = pd.read_csv('../data/cases_borough_weekly.csv')
for col in ['CASE_COUNT','DEATH_COUNT', 'borough_case_count', 'borough_death_count']:
    covid_cases[col] = covid_cases[col].astype(float)
    covid_cases.loc[covid_cases[col]==0,col] = 0.1
    covid_cases[col+'_log'] = np.log(covid_cases[col])

df_4 = df_3.merge(covid_cases, on=['borough', 'week'], how='left')
print(len(df_4))

6477


In [12]:
### 5 merge with policies ####
df_policies = pd.read_csv('../data/stringency_index_ny_wk.csv')
visits_scores_wk = df_4.merge(df_policies, on='week', how='left')
print(len(visits_scores_wk))

6477


In [14]:
visits_scores_wk.to_csv('../data/unpivot_merged_data_raw_loss_visitor.csv',index=False)

In [15]:
visits_scores_wk.columns

Index(['MODZCTA', 'mod_idx', 'week', 'Glocery&Pharmacies_visits_weekly',
       'Retails_visits_weekly', 'Arts&Entertainment_visits_weekly',
       'Restaurants&Bars_visits_weekly', 'Educations_visits_weekly',
       'Healthcares_visits_weekly', 'others_visits_weekly',
       'NEIGHBORHOOD_NAME', 'COVID_CASE_COUNT', 'COVID_CASE_RATE',
       'POP_DENOMINATOR', 'COVID_DEATH_COUNT', 'COVID_DEATH_RATE',
       'PERCENT_POSITIVE', 'TOTAL_COVID_TESTS', 'borough',
       'COVID_CASE_COUNT_log', 'COVID_DEATH_COUNT_log', 'score_mean',
       'score_median', 'score_std', 'gain_bias_mean', 'gain_bias_median',
       'gain_bias_std', 'loss_bias_mean', 'loss_bias_median', 'loss_bias_std',
       'loss_aversion_scores_mean', 'loss_aversion_scores_median',
       'loss_aversion_scores_std', 'rescale_avail_mean',
       'rescale_avail_median', 'rescale_avail_std', 'rescale_realistic_mean',
       'rescale_realistic_median', 'rescale_realistic_std', 'agency_mean',
       'agency_median', 'agency_std',

In [16]:
visits_scores_wk.shape

(6477, 71)

In [13]:
df_zip = pd.read_csv('../results/modzcta_zip_counts.csv')
zip_selected = df_zip[df_zip['Count']>10]['MODZCTA'].values
visits_scores_wk[visits_scores_wk['MODZCTA'].isin(zip_selected)].to_csv('../data/unpivot_merged_data_raw_loss_visitor_s.csv',index=False)